# W4 Friday Project - OOP Calculation of Rental Income (individual project)

Here we assume that we have a client coming to us asking for an automated Rental Property Calculator. Our client's name is Brandon from a company called "Bigger Pockets". Below, you will find a video of what Brandon usually does to calculate his Rental Property ROI.

Using Visual Studio Code/Jupyter Notebook, and Object Oriented Programming create a program that will calculate the Return on Investment(ROI) for a rental property.

This project will be completed individually, though you can feel free to share ideas with your fellow students.

Once completed, commit the project to github and submit the link to this assignment.

also come up with a catch phrase for yourself

In [ ]:
class RentalProperty:
    def __init__(self, purchase_price, rental_income, laundry_income, storage_income, misc_income,
                 taxes, insurance, water_sewer, garbage, electric, gas, hoa_fees, lawn_snow, vacancy,
                 repairs, capex, prop_management, mortgage, down_payment, closing_costs, rehab_budget, misc_other):
        # Initialize the rental property object with the given parameters
        self.purchase_price = purchase_price
        self.rental_income = rental_income
        self.laundry_income = laundry_income
        self.storage_income = storage_income
        self.misc_income = misc_income
        self.taxes = taxes
        self.insurance = insurance
        self.water_sewer = water_sewer
        self.garbage = garbage
        self.electric = electric
        self.gas = gas
        self.hoa_fees = hoa_fees
        self.lawn_snow = lawn_snow
        self.vacancy = vacancy
        self.repairs = repairs
        self.capex = capex
        self.prop_management = prop_management
        self.mortgage = mortgage
        self.down_payment = down_payment
        self.closing_costs = closing_costs
        self.rehab_budget = rehab_budget
        self.misc_other = misc_other

    @classmethod
    def from_user_input(cls):
        print("Enter the property details:")
        purchase_price = float(input("Purchase price of the property: "))
        rental_income = float(input("Monthly rental income: "))
        laundry_income = float(input("Income from laundry facilities: "))
        storage_income = float(input("Income from storage units: "))
        misc_income = float(input("Miscellaneous income: "))

        taxes = float(input("Monthly property taxes: "))
        insurance = float(input("Monthly insurance costs: "))
        water_sewer = float(input("Monthly water and sewer charges: "))
        garbage = float(input("Monthly garbage disposal charges: "))
        electric = float(input("Monthly electric bill: "))
        gas = float(input("Monthly gas bill: "))
        hoa_fees = float(input("Monthly HOA fees: "))
        lawn_snow = float(input("Monthly lawn care and snow removal costs: "))
        vacancy = float(input("Estimated monthly vacancy rate: "))
        repairs = float(input("Estimated monthly repair costs: "))
        capex = float(input("Monthly capital expenditure reserve: "))
        prop_management = float(input("Monthly property management fees: "))

        mortgage = float(input("Monthly mortgage payment: "))
        down_payment = float(input("Initial down payment made on the property: "))
        closing_costs = float(input("Closing costs associated with purchasing the property: "))
        rehab_budget = float(input("Budget for property rehabilitation or renovation: "))
        misc_other = float(input("Miscellaneous other expenses or costs: "))

        return cls(purchase_price, rental_income, laundry_income, storage_income, misc_income,
                   taxes, insurance, water_sewer, garbage, electric, gas, hoa_fees, lawn_snow, vacancy,
                   repairs, capex, prop_management, mortgage, down_payment, closing_costs, rehab_budget, misc_other)

    def calculate_total_monthly_income(self):
        return self.rental_income + self.laundry_income + self.storage_income + self.misc_income

    def calculate_total_monthly_expenses(self):
        return self.taxes + self.insurance + self.water_sewer + self.garbage + self.electric + self.gas + \
               self.hoa_fees + self.lawn_snow + self.vacancy + self.repairs + self.capex + \
               self.prop_management + self.mortgage

    def calculate_cash_flow(self):
        total_monthly_income = self.calculate_total_monthly_income()
        total_monthly_expenses = self.calculate_total_monthly_expenses()
        return (total_monthly_income - total_monthly_expenses) * 12

    def calculate_cash_on_cash_return(self):
        total_investment_cost = self.down_payment + self.closing_costs + self.rehab_budget + self.misc_other
        annual_cash_flow = self.calculate_cash_flow()
        return (annual_cash_flow / total_investment_cost) * 100


# Prompt the user for input and create the rental property object
property1 = RentalProperty.from_user_input()

# Calculate the financial metrics
total_monthly_income = property1.calculate_total_monthly_income()
total_monthly_expenses = property1.calculate_total_monthly_expenses()
cash_flow = property1.calculate_cash_flow()
cash_on_cash_return = property1.calculate_cash_on_cash_return()

# Print the results
print(f"Total Monthly Income: ${total_monthly_income:.2f}")
print(f"Total Monthly Expenses: ${total_monthly_expenses:.2f}")
print(f"Annual Cash Flow: ${cash_flow:.2f}")
print(f"Cash on Cash Return: {cash_on_cash_return:.2f}%")


Once we get an API key, we can use pyzillow for access to real estate information.

In [22]:
import pyzillow
from pyzillow.pyzillow import ZillowWrapper, GetDeepSearchResults
import pandas as pd

house = pd.read_excel('Housing_Output.xlsx')

### Login to Zillow API
address = ['123 Test Street City, State Abbreviation'] # Fill this in with an address
zip_code = ['zip code'] # fill this in with a zip code

# API Key needed from Zillow to login
zillow_data = ZillowWrapper(API KEY)

# Access Zillow deep search API
deep_search_response = zillow_data.get_deep_search_results(address, zip_code)
result = GetDeepSearchResults(deep_search_response)


print(result.zestimate_amount)
print(result.tax_value)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3306486042.py, line 13)